## Using Exceptions to handle deadlock recovery

In [ ]:
CREATE TABLE IF NOT EXISTS foo (a INT, b VARCHAR);
INSERT INTO foo(a, b) VALUES (42, 'The answer.')

---
### Update 

In [ ]:
BEGIN TRANSACTION;
UPDATE foo 
    SET b = 'The answer'
WHERE a = 42;
COMMIT;


In [ ]:
CREATE OR REPLACE FUNCTION update_foo(a INT, b VARCHAR)
    RETURNS VOID
    LANGUAGE plpgsql
    SET lock_timeout = 1000
AS $$    
DECLARE
    retries INT := 10;
    sleep DOUBLE PRECISION := .5;
BEGIN 
    FOR i IN 1..retries LOOP 
        BEGIN
            UPDATE foo 
                SET b = update_foo.b
            WHERE foo.a = update_foo.a;

            RETURN;
        EXCEPTION 
            WHEN lock_not_available THEN
                IF i = retries THEN 
                    RAISE NOTICE 'Update failed due to locking';
                    RAISE;
                    RETURN;
                ELSE
                    RAISE INFO 'Couldn''t get lock, attempt % of %.', i, retries;
                    PERFORM pg_sleep(sleep);
                END IF;
            WHEN OTHERS THEN 
                RAISE;
                RETURN;
        END;
    END LOOP;
END;
$$;

In [ ]:
BEGIN TRANSACTION ISOLATION LEVEL READ COMMITTED;
SELECT update_foo(42, 'The answer');
COMMIT;

---
### Clean up

In [ ]:
DROP TABLE IF EXISTS foo;